- 저서 : 파이썬 생활밀착형 프로젝트 / 백견 불여일타

# 2장. 나에게 카카오톡 메시지 보내기

1. 접근 권한 설정하기
2. api 사용 권한 받기
    1. 인증코드 요청 ->
    2. 인증코드 전달 <-
    3. 인증코드로 토큰 요청 ->
    4. 토큰전달 <-
    5. 토큰으로 api 호출 ->
    6. 토큰 유효성 확인 >> 응답전달 <-
>> 완전 처음이라면 카카오톡 로그인하고 사용동의 + 카카오+open 활용동의 + code를 알 수 있음

https://localhost.com/?code=hrqkQaccTMHTH9HBvDCWsr4qjK1FYH6-C0ve3IRaywtXpUn__un_z3JYANRso0u1jcgMgAoqJQ8AAAGIaef4Sg

In [ ]:
import requests
import pprint

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "",
    "redirect_uri" : "https://localhost.com",
    "code" : ""
}

response = requests.post(url, data=data)

# 요청에 실패했다면,
if response.status_code != 200:
    print("error! because ", response.json())
else: # 성공했다면,
    tokens = response.json()
    pprint.pprint(tokens)

3. token 관리

In [ ]:
import json
import requests
import datetime
import os

# 카카오 토큰을 저장할 파일명
KAKAO_TOKEN_FILENAME = "kakao_message/kakao_token.json"

# 저장하는 함수
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

# 읽어오는 함수
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)

    return tokens

# refresh_token으로 access_token 갱신하는 함수
def update_tokens(app_key, filename) :
    tokens = load_tokens(filename)

    url = "https://kauth.kakao.com/oauth/token"
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    response = requests.post(url, data=data)

    # 요청에 실패했다면,
    if response.status_code != 200:
        print("error! because ", response.json())
        tokens = None
    else: # 성공했다면,
        print(response.json())
        # 기존 파일 백업
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename+"."+ now
        os.rename(filename, backup_filename)
        # 갱신된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# 토큰 저장
save_tokens(KAKAO_TOKEN_FILENAME, tokens)

# 토큰 업데이트 -> 토큰 저장 필수!
# KAKAO_APP_KEY = "" # REST_API 앱키
# tokens = update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)
# save_tokens(KAKAO_TOKEN_FILENAME, tokens)

## 사전 지식
1. 활용할 서비스 확인 : https://developers.kakao.com/docs/latest/ko/message/common#kakaotalk
2. 메시지 종류 : https://developers.kakao.com/docs/latest/ko/message/message-template#type
3. 텍스트와 리스트 메시지 템플릿 확인
    : 텍스트 https://developers.kakao.com/docs/latest/ko/message/message-template#text
        🅐 텍스트: 최대 200자 표시
        🅑 버튼: 최대 2개 표시, 버튼명 8자 이하 권장
    : 리스트 https://developers.kakao.com/docs/latest/ko/message/message-template#list
        🅐 헤더
        🅑 아이템 리스트: 기본 템플릿의 경우 최대 3개, 사용자 정의 템플릿의 경우 최대 5개 목록 표시
        🅒 제목/설명: 최대 3줄 표시 (제목 2줄, 설명 1줄)
        🅓 이미지: 목록별 하나의 이미지 표시, 최소 200x200 픽셀 이상, 2MB이하
        🅔 버튼: 최대 2개 표시, 버튼명 8자 이하 권장

> 메시지를 보낼 수 있는 친구 옵션
- 사업자 정보 등록을 하고 검수 단계를 거친 사용자
- 카카오 개발자 사이트에서 [내어플리케이션] -> 설정 -> 팀 관리에서 추가한 카카오 계정의 친구

## 구현
1. 텍스트 메시지 템플릿을 사용하여 카카오톡 메시지 보내기

In [4]:
import json
import requests

# 저장된 토큰 정보를 읽어 옴
tokens = load_tokens(KAKAO_TOKEN_FILENAME)

# 텍스트 메시지 url
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# request parameter 설정
headers = {
    "Authorization": "Bearer " + tokens['access_token']
}


data = {
    "template_object" : json.dumps({ "object_type" : "text",
                                     "text" : "Hello, world!",
                                     "link" : {
                                         "web_url" : "www.naver.com"
                                     }
                                     })
}

# 나에게 카카오톡 메시지 보내기 요청 (text)
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

# 요청에 실패했다면,
if response.status_code != 200:
    print("error! because ", response.json())
else: # 성공했다면,
    print('메시지를 성공적으로 보냈습니다.')

200
메시지를 성공적으로 보냈습니다.


2. 리스트 메시지 템플릿을 사용하여 카카오촉 메시지 보내기

In [5]:
import json
import requests

# 저장된 토큰 정보를 읽어옴
tokens = load_tokens(KAKAO_TOKEN_FILENAME)

# 리스트 메시지 url
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# request parameter 설정
headers = {
    "Authorization": "Bearer " + tokens['access_token']
}

template = {
    "object_type" : "list",
    "header_title" : "초밥 사진",
    "header_link" : {
        "web_url" : "www.naver.com",
        "mobile_web_url" : "www.naver.com"
    },
    "contents" : [
        {
            "title" : "1. 광어초밥",
            "description" : "광어는 맛있다",
            "image_url" : "https://search1.kakaocdn.net/argon/0x200_85_hr/8x5qcdbcQwi",
            "image_width" : 50, "image_height" : 50,
            "link" : {
                "web_url" : "www.naver.com",
                "mobile_web_url" : "www.naver.com"
            }
        },
        {
            "title" : "2. 참치초밥",
            "description" : "참치는 맛있다",
            "image_url" : "https://search2.kakaocdn.net/argon/0x200_85_hr/IjIToH1S7J1",
            "image_width" : 50, "image_height" : 50,
            "link" : {
                "web_url" : "www.naver.com",
                "mobile_web_url" : "www.naver.com"
            }
        }

    ],
    "buttons" : [
        {
            "title" : "웹으로 이동",
            "link" : {
                "web_url" : "www.naver.com",
                "mobile_web_url" : "www.naver.com"
            }
        }
    ]

}

data = {
    "template_object" : json.dumps(template)
}

# 나에게 카카오톡 메시지 보내기 요청(list)
res = requests.post(url, data=data, headers=headers)
print(response.status_code)

# 요청에 실패했다면,
if response.status_code != 200:
    print("error! because ", response.json())
else: # 성공했다면,
    print('메시지를 성공적으로 보냈습니다.')

200
메시지를 성공적으로 보냈습니다.


## py파일로 함수 저장 후 모듈을 불러서 사용하기

In [ ]:
import kakao_utils

KAKAO_TOKEN_FILENAME = "kakao_message/kakao_token.json"
KAKAO_APP_KEY = "" #<REST_API 앱 키를 입력하세요>
# 토큰 업데이트하기
tokens = kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)
# 업데이트한 토큰 저장하기
save_tokens(KAKAO_TOKEN_FILENAME, tokens)

# 텍스트 템플릿 형식 만들기
template = {
    "object_type": "text",
    "text": "Hello, world!",
    "link": {
        "web_url": "www.naver.com"
    },
}

# 카카오톡 메시지 보내기
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
# 요청에 실패했다면,
if res.status_code != 200:
    print("error! because ", response.json())
else: # 성공했다면,
    print('메시지를 성공적으로 보냈습니다.')

> 코드 구현하면서 더 간결하고 재사용성이 뛰어난 함수와 모듈을 만드는 것이 best

중요정보는 별도 보관해서 사용하는 함수 구현 필수